In [218]:
from sklearn import datasets
from amplpy import AMPL, Environment
import numpy as np

eps = 1e-6
ampl = AMPL(Environment("/Users/alex/Desktop/ampl_macosx64"))

In [219]:
## Training pel primal
def primal_tr(model,data):
    ampl.reset()
    ampl.read(model)
    ampl.readData(data)
    ampl.setOption('solver','minos')
    ampl.solve()
    
    # Parametres
    nu = ampl.getParameter('nu').getValues().toList()[0]
    m = int(ampl.getParameter('m').getValues().toList()[0])
    n = int(ampl.getParameter('n').getValues().toList()[0])
    A = np.matrix(ampl.getParameter('A').getValues().toPandas()).reshape(m,n)
    y = np.matrix(ampl.getParameter('y').getValues().toPandas()).reshape(m,1)
    # Variables
    w = np.matrix(ampl.getVariable('w').getValues().toPandas()).reshape(n,1)
    gamma = ampl.getVariable('gamma').getValues().toList()[0]
    s = np.matrix(ampl.getVariable('s').getValues().toPandas()).reshape(m,1)
    
    # Training accuracy primal
    
    wt = w.T
    prediccio = wt*A.T+gamma
    pred = prediccio.T

    ok = 0
    for i in range(m):
        if y[i]==1:
            if pred[i]>+eps:
                ok = ok + 1
        else: # y = -1
            if pred[i]<-eps:
                ok = ok + 1
            
    tr_acc = ok/m*100
    return tr_acc,w,gamma

In [220]:
## Test pel primal
def primal_te(model,data,w,gamma):
    ampl.reset()
    ampl.read(model)
    ampl.readData(data)

    nu = ampl.getParameter('nu').getValues().toList()[0]
    m = int(ampl.getParameter('m').getValues().toList()[0])
    n = int(ampl.getParameter('n').getValues().toList()[0])
    A = np.matrix(ampl.getParameter('A').getValues().toPandas()).reshape(m,n)
    y = np.matrix(ampl.getParameter('y').getValues().toPandas()).reshape(m,1)
    
    # Test accuracy primal
    
    wt = w.T
    prediccio = wt*A.T+gamma
    pred = prediccio.T

    ok = 0
    for i in range(m):
        if y[i]==1:
            if pred[i]>-eps:
                ok = ok + 1
        else: # y = -1
            if pred[i]<-eps:
                ok = ok + 1
            
    te_acc = ok/m*100
    return te_acc

In [221]:
def find_gamma(y,landa,K,nu,num_points):
    #In order to obtain the gamma, first we find a SV point.
    SV = 0
    for i in range (num_points):
        if (eps < landa[i] < nu-eps):
                SV = i;
                break;

    #Then we proceed to calculate the gamma.
    gamma = y[SV]
    for j in range(num_points):
        gamma = gamma - landa[j]*y[j]*K[SV,j]
    return gamma

In [222]:
def find_w(y,landa,A,num_points):
    w = [0 for i in range(A[0].size)]

    for i in range (num_points):
        aux = landa[i]*y[i]*A[i,:]
        w += aux

    w = np.matrix(w)
    w = w.reshape(w.size, 1)

    return w

In [223]:
## Training pel dual
def dual_tr(model,data):
    ampl.reset()
    ampl.read(model)
    ampl.readData(data)
    ampl.setOption('solver','minos')
    ampl.solve()

   # Parametres
    nu = ampl.getParameter('nu').getValues().toList()[0]
    n = int(ampl.getParameter('n').getValues().toList()[0])
    m = int(ampl.getParameter('m').getValues().toList()[0])
    K = np.matrix(ampl.getParameter('K').getValues().toPandas()).reshape(m,m)
    y = np.matrix(ampl.getParameter('y').getValues().toPandas()).reshape(m,1)
    A = np.matrix(ampl.getParameter('A').getValues().toPandas()).reshape(m,n)
    # Variables
    landa = np.matrix(ampl.getVariable('lambda').getValues().toPandas()).reshape(m,1)
    
    # Training accuracy dual
    gamma = find_gamma(y,landa,K,nu,m)
    print(gamma)
    ok = 0
    for j in range(m):
        suma = 0
        for i in range(m):
            suma = suma + landa[i]*y[i]*K[i,j]
        pred = suma + gamma        
        if y[j]==1:
            if pred>=+eps:
                ok = ok + 1
        else: # y = -1
            if pred<-eps:
                ok = ok + 1
    
    tr_acc = ok/m*100
    
    # Trobar w per despres pel test
    w = find_w(y,landa,A,m)    
    
    return tr_acc,w,gamma

In [224]:
## Test pel dual
def dual_te(model,data,w,gamma):
    ampl.reset()
    ampl.read(model)
    ampl.readData(data)

    nu = ampl.getParameter('nu').getValues().toList()[0]
    n = int(ampl.getParameter('n').getValues().toList()[0])
    m = int(ampl.getParameter('m').getValues().toList()[0])
    K = np.matrix(ampl.getParameter('K').getValues().toPandas()).reshape(m,m)
    y = np.matrix(ampl.getParameter('y').getValues().toPandas()).reshape(m,1)
    A = np.matrix(ampl.getParameter('A').getValues().toPandas()).reshape(m,n)
    
    wt = w.T
    prediccio = wt*A.T+gamma
    pred = prediccio.T
    
    ok = 0
    for i in range(m):
        if y[i]==1:
            if pred[i]>+eps:
                ok = ok + 1
        else: # y = -1
            if pred[i]<-eps:
                ok = ok + 1
            
    tr_acc = ok/m*100
    return tr_acc



In [225]:
###############################################################

In [226]:
# Primal - training
model = "./Desktop/2n/OM/Practica2/svm_primal.mod"
data = "./Desktop/2n/OM/Practica2/primal_train.dat"
tr_acc_primal,w_primal,gamma = primal_tr(model,data)
tr_acc_primal,gamma

MINOS 5.51: optimal solution found.
91 iterations, objective 25.91837833
Nonlin evals: obj = 102, grad = 101.


(91.0, -3.6579280069385725)

In [227]:
# Primal -  test
model = "./Desktop/2n/OM/Practica2/svm_primal.mod"
data = "./Desktop/2n/OM/Practica2/primal_test.dat"
te_acc_primal = primal_te(model,data,w_primal,gamma)
te_acc_primal

88.4

In [125]:
# Dual - training
model = "./Desktop/2n/OM/Practica2/svm_dual.mod"
data = "./Desktop/2n/OM/Practica2/dual_train.dat"
tr_acc_dual,w_dual,gamma = dual_tr(model,data)
tr_acc_dual

MINOS 5.51: optimal solution found.
71 iterations, objective 25.91837833
Nonlin evals: obj = 81, grad = 80.
[[-3.65792769]]


91.0

In [126]:
# Dual - test
data = "./Desktop/2n/OM/Practica2/dual_test.dat"
te_acc_dual = dual_te(model,data,w_dual,gamma)
te_acc_dual

88.4

In [127]:
# Comparació hyperplans primal vs dual:
print(w_primal,"\n\n",w_dual)

[[1.73188892]
 [1.82189662]
 [1.82182107]
 [1.655764  ]] 

 [[1.73188877]
 [1.82189716]
 [1.82182025]
 [1.65576422]]


In [128]:
## Dataset d'un altre repositori ##
# Classificació d'un tipus d'iris en funció de 4 característiques
# "sepal length" "sepal width" "petal length" "petal width" "y"

In [129]:
# Primal - training iris1
model = "./Desktop/2n/OM/Practica2/svm_primal.mod"
data = "./Desktop/2n/OM/Practica2/iris1_primal.dat"
tr_acc_primal,w_iris1_primal,gamma1 = primal_tr(model,data)
tr_acc_primal,gamma1

MINOS 5.51: optimal solution found.
166 iterations, objective 0.6960803358
Nonlin evals: obj = 176, grad = 175.


(100.0, 1.380028786875222)

In [130]:
# Primal - training iris2
model = "./Desktop/2n/OM/Practica2/svm_primal.mod"
data = "./Desktop/2n/OM/Practica2/iris2_primal.dat"
tr_acc_primal,w_iris2_primal,gamma2 = primal_tr(model,data)
tr_acc_primal,gamma2

MINOS 5.51: optimal solution found.
118 iterations, objective 45.9851966
Nonlin evals: obj = 122, grad = 121.


(74.0, 4.398232853094191)

In [131]:
# Primal - training iris3
model = "./Desktop/2n/OM/Practica2/svm_primal.mod"
data = "./Desktop/2n/OM/Practica2/iris3_primal.dat"
tr_acc_primal,w_iris3_primal,gamma3 = primal_tr(model,data)
tr_acc_primal,gamma3

MINOS 5.51: optimal solution found.
197 iterations, objective 9.903586036
Nonlin evals: obj = 201, grad = 200.


(98.66666666666667, -7.564276080920203)

In [132]:
# Dual - training iris1
model = "./Desktop/2n/OM/Practica2/svm_dual.mod"
data = "./Desktop/2n/OM/Practica2/iris1_dual.dat"
tr_acc_dual,w_iris1_dual,gamma = dual_tr(model,data)
tr_acc_dual

MINOS 5.51: optimal solution found.
17 iterations, objective 0.6960803358
Nonlin evals: obj = 34, grad = 33.
[[1.38002879]]


100.0

In [133]:
# Dual - training iris2
model = "./Desktop/2n/OM/Practica2/svm_dual.mod"
data = "./Desktop/2n/OM/Practica2/iris2_dual.dat"
tr_acc_dual,w_iris2_dual,gamma = dual_tr(model,data)
tr_acc_dual

MINOS 5.51: optimal solution found.
215 iterations, objective 45.9851966
Nonlin evals: obj = 361, grad = 360.
[[4.39823285]]


74.0

In [134]:
# Dual - training iris3
model = "./Desktop/2n/OM/Practica2/svm_dual.mod"
data = "./Desktop/2n/OM/Practica2/iris3_dual.dat"
tr_acc_dual,w_iris3_dual,gamma = dual_tr(model,data)
tr_acc_dual

MINOS 5.51: optimal solution found.
42 iterations, objective 9.903586036
Nonlin evals: obj = 60, grad = 59.
[[-7.56427462]]


98.66666666666667

In [135]:
# Comprovem que donen plans de separació iguals:
print("W iris1: \n",w_iris1_primal,"   \n\n",w_iris1_dual,"\n")
print("W iris2: \n",w_iris2_primal,"   \n\n",w_iris2_dual,"\n")
print("W iris3: \n",w_iris3_primal,"   \n\n",w_iris3_dual,"\n")
print("Gamma iris1: \n", gamma1,"   \n",gamma1,"\n")
print("Gamma iris2: \n",gamma2,"   \n",gamma2,"\n")
print("Gamma iris3: \n",gamma3,"   \n",gamma3,"\n")

W iris1: 
 [[-0.0355123 ]
 [ 0.45138075]
 [-0.87612727]
 [-0.39811229]]    

 [[-0.0355123 ]
 [ 0.45138075]
 [-0.87612728]
 [-0.3981123 ]] 

W iris2: 
 [[ 0.00909582]
 [-1.79942868]
 [ 0.36151845]
 [-0.94013852]]    

 [[ 0.00909582]
 [-1.79942868]
 [ 0.36151845]
 [-0.94013852]] 

W iris3: 
 [[-0.47356568]
 [-0.46601908]
 [ 1.83651599]
 [ 1.70013342]]    

 [[-0.47356587]
 [-0.46601907]
 [ 1.83651578]
 [ 1.70013395]] 

Gamma iris1: 
 1.380028786875222    
 1.380028786875222 

Gamma iris2: 
 4.398232853094191    
 4.398232853094191 

Gamma iris3: 
 -7.564276080920203    
 -7.564276080920203 



In [171]:
## Training pel dual
def dual_tr_rbf(model,data):
    ampl.reset()
    ampl.read(model)
    ampl.readData(data)
    ampl.setOption('solver','minos')
    ampl.solve()

   # Parametres
    nu = ampl.getParameter('nu').getValues().toList()[0]
    n = int(ampl.getParameter('n').getValues().toList()[0])
    m = int(ampl.getParameter('m').getValues().toList()[0])
    K = np.matrix(ampl.getParameter('K').getValues().toPandas()).reshape(m,m)
    y = np.matrix(ampl.getParameter('y').getValues().toPandas()).reshape(m,1)
    A = np.matrix(ampl.getParameter('A').getValues().toPandas()).reshape(m,n)
    # Variables
    landa = np.matrix(ampl.getVariable('lambda').getValues().toPandas()).reshape(m,1)
    
    # Training accuracy dual
    gamma = find_gamma(y,landa,K,nu,m)
    print(gamma)
    ok = 0
    for j in range(m):
        suma = 0
        for i in range(m):
            suma = suma + landa[i]*y[i]*K[i,j]
        pred = suma + gamma        
        if y[j]==1:
            if pred>=+eps:
                ok = ok + 1
        else: # y = -1
            if pred<-eps:
                ok = ok + 1
    
    tr_acc = ok/m*100
    
    return tr_acc,landa,gamma,y,A

In [172]:
def kernel(A1, A2):
    sig2 = (A1[:,0].var() + A1[:,1].var() + A1[:,2].var())/3
    m = len(A1) #assumim que len(A1) = len(A2)
    
    K = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            K[i,j] = np.exp(-np.linalg.norm(A1[i,:] - A2[j,:])**2/sig2/2)
    K = K + np.eye(m)*eps
    
    return K

In [215]:
## Test pel dual
def dual_te_rbf(model,data,landa,gamma,y_tr,A_tr):
    ampl.reset()
    ampl.read(model)
    ampl.readData(data)

    nu = ampl.getParameter('nu').getValues().toList()[0]
    n = int(ampl.getParameter('n').getValues().toList()[0])
    m = int(ampl.getParameter('m').getValues().toList()[0])
    y = np.matrix(ampl.getParameter('y').getValues().toPandas()).reshape(m,1)
    A_te = np.matrix(ampl.getParameter('A').getValues().toPandas()).reshape(m,n)
    
    K = kernel(A_tr,A_te)
    
    pred = [1 for i in range(m)]
    for i in range(m):
        suma = 0
        for j in range(m):
            suma += landa[j]*y_tr[j]*K[j,i]
        if suma + gamma < - eps: pred[i] = -1
    
    ok = 0
    for i in range(m):
        if pred[i] == y[i]:
            ok += 1
            
    tr_acc = ok/m*100
    return tr_acc

In [228]:
## RBF
# Training
model = "./Desktop/2n/OM/Practica2/svm_dual.mod"
data = "./Desktop/2n/OM/Practica2/rbf_train.dat"
tr_acc_dual,landa,gamma,y_tr,A_tr = dual_tr_rbf(model,data)
tr_acc_dual

OSError: Can't find  file "Desktop/2n/OM/Practica2/svm_dual.mod"

In [217]:
# Test
data = "./Desktop/2n/OM/Practica2/rbf_test.dat"
te_acc_dual = dual_te_rbf(model,data,landa,gamma,y_tr,A_tr)
te_acc_dual

99.0